In [ ]:
#Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import PredefinedSplit
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from mrmr import mrmr_regression as mrmr_reg
random_seed = 10
np.set_printoptions(suppress=True)
np.random.seed(random_seed)

In [ ]:
#Reading in gene set score 
set_df = pd.read_csv('set_scores.csv')

In [ ]:
#Subsetting gene set PRS scores only
prs_info = set_df.iloc[:,:65]

In [ ]:
#Plotting Gene Set Correlation Matrix
sns.set_style('darkgrid', {'axes.linewidth': 1, 'axes.edgecolor':'black'})
corr_matrix = prs_info.corr().round(2)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.set(rc = {'figure.figsize':(15,8)})
corr_plot = sns.heatmap(corr_matrix, vmax=1, vmin=-1, center=0, cmap='viridis', mask=mask)
corr_plot.set_title('Gene Set Correlation Matrix', weight='bold')
corr_plot.set_xlabel("", weight='bold')
corr_plot.set_ylabel("", weight='bold')
sns.set(font_scale = 1.6)
sns.despine(top=False, right=False, left=False, bottom=False)
plt.show()
corr_plot.figure.savefig("gene_set_corr.jpg")

In [ ]:
#Reading in each datasets for each round of 5-fold CV 
#NOTE: PCs for validation sets are projections onto components from training folds (these are pre-computed)
folds_1 = pd.read_csv('folds_1.csv')
folds_2 = pd.read_csv('folds_2.csv')
folds_3 = pd.read_csv('folds_3.csv')
folds_4 = pd.read_csv('folds_4.csv')
folds_5 = pd.read_csv('folds_5.csv')

folds_list = [folds_1, folds_2, folds_3, folds_4, folds_5]

In [ ]:
#Appending Gene Set Scores to each of the datasets 

#Dict to store updated folds
fold_dict = {'fold_1': 0, 'fold_2':0, 'fold_3':0, 'fold_4':0, 'fold_5':0}

for i, fold in enumerate(folds_list):
    #Retaining only PCs
    fold_pcs = fold[['PC1','PC2', 'PC3', 'PC4', 'PC5']]
    temp_data = pd.concat([fold_pcs, set_df], axis = 1)
    fold_str = f'fold_{i+1}'
    fold_dict[fold_str] = temp_data

In [ ]:
#Read in IDs for train-val folds
#Dict to store validation fold ids
val_id_dict = {'val_1': 0, 'val_2':0, 'val_3':0, 'val_4':0, 'val_5':0}

for i in range(5):
    #Adding validation set IDs to dictionary for each CV round
    id_str = f'val_{i+1}'
    val_id_dict[id_str] = pd.read_csv(f'fold_{i+1}.txt', header=None).iloc[:,0]

In [ ]:
#Creating data and spits for each separate round of 5-fold CV
x_1, y_1 = fold_dict['fold_1'].iloc[:,:-1], fold_dict['fold_1'].iloc[:,-1]
split_1 = [0 if x in val_id_dict['val_1'].values else -1 for x in x_1.index]
ps_1 = PredefinedSplit(split_1)

x_2, y_2 = fold_dict['fold_2'].iloc[:,:-1], fold_dict['fold_2'].iloc[:,-1]
split_2 = [0 if x in val_id_dict['val_2'].values else -1 for x in x_2.index]
ps_2 = PredefinedSplit(split_2)

x_3, y_3 = fold_dict['fold_3'].iloc[:,:-1], fold_dict['fold_3'].iloc[:,-1]
split_3 = [0 if x in val_id_dict['val_3'].values else -1 for x in x_3.index]
ps_3 = PredefinedSplit(split_3)

x_4, y_4 = fold_dict['fold_4'].iloc[:,:-1], fold_dict['fold_4'].iloc[:,-1]
split_4 = [0 if x in val_id_dict['val_4'].values else -1 for x in x_4.index]
ps_4 = PredefinedSplit(split_4)

x_5, y_5 = fold_dict['fold_5'].iloc[:,:-1], fold_dict['fold_5'].iloc[:,-1]
split_5 = [0 if x in val_id_dict['val_5'].values else -1 for x in x_5.index]
ps_5 = PredefinedSplit(split_5)

$Feature Selection$

In [ ]:
#Return list of top 10, 15, 20, 25, 30 features from MRMR feature selection
def get_top_features(x, y, val_ids):
    
    #Removing principal components
    new_data = x.iloc[:,5:]
    
    #Top 50 features using training data only
    top_features = mrmr_reg(new_data.loc[~new_data.index.isin(val_ids.values)], y.loc[~y.index.isin(val_ids.values)].values, K=50)
    
    #Subsetting based on top N features
    x_10 = pd.concat([new_data.loc[:,top_features[:10]], x.iloc[:,:5]], axis=1)
    x_15 = pd.concat([new_data.loc[:,top_features[:15]], x.iloc[:,:5]], axis=1)
    x_20 = pd.concat([new_data.loc[:,top_features[:20]], x.iloc[:,:5]], axis=1)
    x_25 = pd.concat([new_data.loc[:,top_features[:25]], x.iloc[:,:5]], axis=1)
    x_30 = pd.concat([new_data.loc[:,top_features[:30]], x.iloc[:,:5]], axis=1)
    
    return [x_10, x_15, x_20, x_25, x_30]

In [ ]:
#List of feature sets to test
num_feats = ['10', '15', '20', '25', '30']

In [ ]:
#Get list of top features for each dataset for each round of CV
x1_list = get_top_features(x_1, y_1, val_id_dict['val_1'])
x2_list = get_top_features(x_2, y_2, val_id_dict['val_2'])
x3_list = get_top_features(x_3, y_3, val_id_dict['val_3'])
x4_list = get_top_features(x_4, y_4, val_id_dict['val_4'])
x5_list = get_top_features(x_5, y_5, val_id_dict['val_5'])

In [ ]:
#Function to return dataframe of 5-fold CV results
def cv_results(result_list):
    result_dict = {'params':result_list[0]['params']}
    
    #Storing results from each CV round
    for i, result in enumerate(result_list):
        result_dict[f'split_{i}_neg_mse'] = result_list[i]['mean_test_neg_mean_squared_error']
        result_dict[f'split_{i}_r2'] = result_list[i]['mean_test_r2']
    
    #Calculating mean and std of results
    num_res = len(result_dict['params'])
                  
    mean_mse_list = []
    sd_mse_list = []
    mean_r2_list = []
    sd_r2_list = []
                  
    for j in range(num_res):
        mse_vals = [result_dict['split_0_neg_mse'][j], result_dict['split_1_neg_mse'][j], result_dict['split_2_neg_mse'][j],
                           result_dict['split_3_neg_mse'][j], result_dict['split_4_neg_mse'][j]]
        r2_vals = [result_dict['split_0_r2'][j], result_dict['split_1_r2'][j], result_dict['split_2_r2'][j],
                           result_dict['split_3_r2'][j], result_dict['split_4_r2'][j]]
        
        mean_mse = np.mean(mse_vals)
        sd_mse = np.std(mse_vals)
        mean_r2 = np.mean(r2_vals)
        sd_r2 = np.std(r2_vals)
        
        mean_mse_list.append(mean_mse)
        sd_mse_list.append(sd_mse)
        mean_r2_list.append(mean_r2)
        sd_r2_list.append(sd_r2)
        
    result_dict['mean_neg_mse'] = mean_mse_list
    result_dict['sd_neg_mse'] = sd_mse_list
    result_dict['mean_r2'] = mean_r2_list
    result_dict['sd_r2'] = sd_r2_list
    
    return result_dict

$Feature Scaling$

In [ ]:
#Returns list of scaled dataset for each set of features
def scale_data(x, y, val_dict, val_id):
    
    scaler = StandardScaler()
    #Rescaling sex
    x['sex'] = x['sex'].apply(lambda x: -1 if x == 1 else 1)
    
    #Scaling remaining features
    x_new = scaler.fit_transform(x.iloc[:,:70])
    x_new = pd.DataFrame(data=x_new, columns=scaler.get_feature_names_out() )
    
    #Joining datasets
    x_new = pd.concat([x_new, x.iloc[:,-1]], axis=1)
    
    #Get list of datasets with top 10, 15, 20, 25, and 30 features
    x_list = get_top_features(x_new, y, val_dict[val_id])
    
    return x_list 

In [ ]:
#Creating scaled datasets
x1_scale = scale_data(x_1, y_1, val_id_dict, 'val_1')
x2_scale = scale_data(x_2, y_2, val_id_dict, 'val_2')
x3_scale = scale_data(x_3, y_3, val_id_dict, 'val_3')
x4_scale = scale_data(x_4, y_4, val_id_dict, 'val_4')
x5_scale = scale_data(x_5, y_5, val_id_dict, 'val_5')

$Elastic Net Regression$

In [ ]:
#Parameter Search Grid for Elastic Net Model
param_grid_enet = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
              "l1_ratio": [.1, .5, .7, .9, .95, .99, 1]}

#Defining Model
eNet = ElasticNet()

In [ ]:
#Reduced Feature Sets
#Carry out each round of 5-fold CV separately to ensure pre-defined folds are used
#Dict to store results for each feature set
enet_result_dict = {'10':0, '15':0, '20':0, '25':0, '30':0}

#Get results for each reduced set of features
for i, feats in enumerate(num_feats):
    grid_enet_1 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_1, refit=False)
    grid_enet_1.fit(x1_list[i], y_1)

    grid_enet_2 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_2, refit=False)
    grid_enet_2.fit(x2_list[i], y_2)

    grid_enet_3 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_3, refit=False)
    grid_enet_3.fit(x3_list[i], y_3)

    grid_enet_4 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_4, refit=False)
    grid_enet_4.fit(x4_list[i], y_4)

    grid_enet_5 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_5, refit=False)
    grid_enet_5.fit(x5_list[i], y_5)

    enet_result_list = [grid_enet_1.cv_results_, grid_enet_2.cv_results_, grid_enet_3.cv_results_, grid_enet_4.cv_results_, 
                        grid_enet_5.cv_results_]

    enet_result_dict[feats] = cv_results(enet_result_list)

In [ ]:
#Full feature set
grid_enet_1 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_1, refit=False)
grid_enet_1.fit(x_1, y_1)

grid_enet_2 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_2, refit=False)
grid_enet_2.fit(x_2, y_2)

grid_enet_3 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_3, refit=False)
grid_enet_3.fit(x_3, y_3)

grid_enet_4 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_4, refit=False)
grid_enet_4.fit(x_4, y_4)

grid_enet_5 = GridSearchCV(eNet, param_grid_enet, scoring=['neg_mean_squared_error','r2'], cv=ps_5, refit=False)
grid_enet_5.fit(x_5, y_5)

enet_result_list = [grid_enet_1.cv_results_, grid_enet_2.cv_results_, grid_enet_3.cv_results_, grid_enet_4.cv_results_, 
                    grid_enet_5.cv_results_]

enet_full_set_results = cv_results(enet_result_list)

In [ ]:
with open('enet_full_set_results_final.pkl', 'wb') as f:
    pickle.dump(enet_full_set_results, f)
    
with open('enet_feat_sets_results_final.pkl', 'wb') as f:
    pickle.dump(enet_result_dict, f)

$XGBoost$

In [ ]:
param_grid_xgb = {'n_estimators': [50, 100, 250, 500, 1000, 1500],
    'learning_rate': [0.2, 0.05, 0.01, 0.001],
    'max_depth': [3, 6, 8],
    'min_child_weight': [3, 6],
    'gamma': [0.2, 0.5],
    'subsample': [0.5, 0.8],
    'random_state': [1],
    'colsample_bytree':[0.5,0.8],
    'n_jobs': [-1],
    'objective': ['reg:squarederror']}

xgb_model = XGBRegressor()

In [ ]:
#Reduced Feature Sets
#Carry out each round of 5-fold CV separately to ensure pre-defined folds are used
#Dict to store results for each feature set
xgb_result_dict = {'10':0, '15':0, '20':0, '25':0, '30':0}

#Get results for each reduced set of features
for i, feats in enumerate(num_feats):
    
    print(f'Feature set {i+1}/5')
    grid_xgb_1 = GridSearchCV(xgb_model, param_grid_xgb, scoring=['neg_mean_squared_error','r2'], cv=ps_1, refit=False)
    grid_xgb_1.fit(x1_list[i], y_1)

    grid_xgb_2 = GridSearchCV(xgb_model, param_grid_xgb, scoring=['neg_mean_squared_error','r2'], cv=ps_2, refit=False)
    grid_xgb_2.fit(x2_list[i], y_2)

    grid_xgb_3 = GridSearchCV(xgb_model, param_grid_xgb, scoring=['neg_mean_squared_error','r2'], cv=ps_3, refit=False)
    grid_xgb_3.fit(x3_list[i], y_3)

    grid_xgb_4 = GridSearchCV(xgb_model, param_grid_xgb, scoring=['neg_mean_squared_error','r2'], cv=ps_4, refit=False)
    grid_xgb_4.fit(x4_list[i], y_4)

    grid_xgb_5 = GridSearchCV(xgb_model, param_grid_xgb, scoring=['neg_mean_squared_error','r2'], cv=ps_5, refit=False)
    grid_xgb_5.fit(x5_list[i], y_5)

    xgb_result_list = [grid_xgb_1.cv_results_, grid_xgb_2.cv_results_, grid_xgb_3.cv_results_, grid_xgb_4.cv_results_, 
                        grid_xgb_5.cv_results_]

    xgb_result_dict[feats] = cv_results(xgb_result_list)

In [ ]:
with open('xgb_results_final.pkl', 'wb') as f:
    pickle.dump(xgb_result_dict, f)

In [ ]:
#Further tuning of regularisation terms
param_grid_xgb_reg = {'n_estimators': [500, 1000, 1500],
    'learning_rate': [0.05, 0.01, 0.001],
    'max_depth': [3, 6],
    'min_child_weight': [3, 6],
    'gamma': [0.2, 0.5],
    'subsample': [0.5, 0.8],
    'random_state': [1],
    'colsample_bytree':[0.5, 0.8],
    'n_jobs': [-1],
    'objective': ['reg:squarederror'],
    'reg_lambda':[0.01, 0.1, 1, 10, 100],
    'reg_alpha':[0.01, 0.1, 1, 10, 100]}

xgb_model_reg = XGBRegressor()

In [ ]:
grid_xgb_1_reg = GridSearchCV(xgb_model_reg, param_grid_xgb_reg, scoring=['neg_mean_squared_error','r2'], cv=ps_1, refit=False)
grid_xgb_1_reg.fit(x_1, y_1)

grid_xgb_2_reg = GridSearchCV(xgb_model_reg, param_grid_xgb_reg, scoring=['neg_mean_squared_error','r2'], cv=ps_2, refit=False)
grid_xgb_2_reg.fit(x_2, y_2)

grid_xgb_3_reg = GridSearchCV(xgb_model_reg, param_grid_xgb_reg, scoring=['neg_mean_squared_error','r2'], cv=ps_3, refit=False)
grid_xgb_3_reg.fit(x_3, y_3)

grid_xgb_4_reg = GridSearchCV(xgb_model_reg, param_grid_xgb_reg, scoring=['neg_mean_squared_error','r2'], cv=ps_4, refit=False)
grid_xgb_4_reg.fit(x_4, y_4)

grid_xgb_5_reg = GridSearchCV(xgb_model_reg, param_grid_xgb_reg, scoring=['neg_mean_squared_error','r2'], cv=ps_5, refit=False)
grid_xgb_5_reg.fit(x_5, y_5)

xgb_result_list_reg = [grid_xgb_1_reg.cv_results_, grid_xgb_2_reg.cv_results_, grid_xgb_3_reg.cv_results_, grid_xgb_4_reg.cv_results_, 
                    grid_xgb_5_reg.cv_results_]

xgb_full_set_results = cv_results(xgb_result_list_reg)

In [ ]:
with open('xgb_results_final_full_set.pkl', 'wb') as f:
    pickle.dump(xgb_full_set_results, f)

$Random Forest$

In [ ]:
#Feature Selection Grid
param_grid_rf = {'n_estimators': [100, 250, 500, 1000, 1500, 2000],
                 'criterion':['squared_error'],
                 'max_features':[2, 4, 6, 8],
                 'max_depth':[int(x) for x in np.linspace(10, 100, num = 10)],
                 'min_samples_split':[2, 5, 10],
                 'min_samples_leaf':[1, 2, 4]
                 }

rf_model = RandomForestRegressor()

In [ ]:
#Reduced Feature Sets
#Carry out each round of 5-fold CV separately to ensure pre-defined folds are used
#Dict to store results for each feature set
rf_result_dict = {'10':0, '15':0, '20':0, '25':0, '30':0}

#Get results for each reduced set of features
for i, feats in enumerate(num_feats):
    
    print(f'Feature set {i+1}/5')
    
    #Carry out each round of 5-fold CV separately to ensure pre-defined folds are used
    grid_rf_1 = GridSearchCV(rf_model, param_grid_rf, scoring=['neg_mean_squared_error','r2'], cv=ps_1, refit=False)
    grid_rf_1.fit(x1_list[i], y_1)

    grid_rf_2 = GridSearchCV(rf_model, param_grid_rf, scoring=['neg_mean_squared_error','r2'], cv=ps_2, refit=False)
    grid_rf_2.fit(x2_list[i], y_2)

    grid_rf_3 = GridSearchCV(rf_model, param_grid_rf, scoring=['neg_mean_squared_error','r2'], cv=ps_3, refit=False)
    grid_rf_3.fit(x3_list[i], y_3)

    grid_rf_4 = GridSearchCV(rf_model, param_grid_rf, scoring=['neg_mean_squared_error','r2'], cv=ps_4, refit=False)
    grid_rf_4.fit(x4_list[i], y_4)

    grid_rf_5 = GridSearchCV(rf_model, param_grid_rf, scoring=['neg_mean_squared_error','r2'], cv=ps_5, refit=False)
    grid_rf_5.fit(x5_list[i], y_5)
    
    rf_result_list = [grid_rf_1.cv_results_, grid_rf_2.cv_results_, grid_rf_3.cv_results_, grid_rf_4.cv_results_, 
                        grid_rf_5.cv_results_]

    rf_result_dict[feats] = cv_results(rf_result_list)

In [ ]:
with open('rf_results_final.pkl', 'wb') as f:
    pickle.dump(rf_result_dict, f)

In [ ]:
#Full Feature Set Grid
param_grid_rf_full = {'n_estimators': [100, 250, 500, 1000, 1500, 2000],
                 'criterion':['squared_error'],
                 'max_features':[4, 6, 8, 10],
                 'max_depth':[int(x) for x in np.linspace(10, 100, num = 10)],
                 'min_samples_split':[2, 5, 10],
                 'min_samples_leaf':[1, 2, 4]
                 }

rf_model = RandomForestRegressor()

In [ ]:
#Full Feature Set
#Carry out each round of 5-fold CV separately to ensure pre-defined folds are used

#Get results for each reduced set of features
#Carry out each round of 5-fold CV separately to ensure pre-defined folds are used
grid_rf_full_set_1 = GridSearchCV(rf_model, param_grid_rf_full, scoring=['neg_mean_squared_error','r2'], cv=ps_1, refit=False)
grid_rf_full_set_1.fit(x_1, y_1)

grid_rf_full_set_2 = GridSearchCV(rf_model, param_grid_rf_full, scoring=['neg_mean_squared_error','r2'], cv=ps_2, refit=False)
grid_rf_full_set_2.fit(x_2, y_2)

grid_rf_full_set_3 = GridSearchCV(rf_model, param_grid_rf_full, scoring=['neg_mean_squared_error','r2'], cv=ps_3, refit=False)
grid_rf_full_set_3.fit(x_3, y_3)

grid_rf_full_set_4 = GridSearchCV(rf_model, param_grid_rf_full, scoring=['neg_mean_squared_error','r2'], cv=ps_4, refit=False)
grid_rf_full_set_4.fit(x_4, y_4)

grid_rf_full_set_5 = GridSearchCV(rf_model, param_grid_rf_full, scoring=['neg_mean_squared_error','r2'], cv=ps_5, refit=False)
grid_rf_full_set_5.fit(x_5, y_5)

rf_full_set_result_list = [grid_rf_full_set_1.cv_results_, grid_rf_full_set_2.cv_results_, grid_rf_full_set_3.cv_results_, grid_rf_full_set_4.cv_results_, 
                    grid_rf_full_set_5.cv_results_]

rf_full_set_results = cv_results(rf_full_set_result_list) 

In [ ]:
with open('rf_results_full_set_final.pkl', 'wb') as f:
    pickle.dump(rf_full_set_results, f)

$Support Vector Machine$

In [ ]:
param_grid_svm = {'kernel': ['linear', 'poly', 'rbf'],
                 'degree':[2,3,4],
                 'gamma':['auto', 'scale'],
                 'C':[0.1, 1, 10, 100, 1000],
                 'epsilon':[0, 0.01, 0.1, 0.5, 1, 2, 5]
                 }

svm_model = SVR()

In [ ]:
#Reduced Feature Sets
#Carry out each round of 5-fold CV separately to ensure pre-defined folds are used
#Dict to store results for each feature set
svm_result_dict = {'10':0, '15':0, '20':0, '25':0, '30':0}

#Get results for each reduced set of features
for i, feats in enumerate(num_feats):
    
    print(f'Feature set {i+1}/5')
    
    #Carry out each round of 5-fold CV separately to ensure pre-defined folds are used
    grid_svm_1 = GridSearchCV(svm_model, param_grid_svm, scoring=['neg_mean_squared_error','r2'], cv=ps_1, refit=False)
    grid_svm_1.fit(x1_scale[i], y_1)

    grid_svm_2 = GridSearchCV(svm_model, param_grid_svm, scoring=['neg_mean_squared_error','r2'], cv=ps_2, refit=False)
    grid_svm_2.fit(x2_scale[i], y_2)

    grid_svm_3 = GridSearchCV(svm_model, param_grid_svm, scoring=['neg_mean_squared_error','r2'], cv=ps_3, refit=False)
    grid_svm_3.fit(x3_scale[i], y_3)

    grid_svm_4 = GridSearchCV(svm_model, param_grid_svm, scoring=['neg_mean_squared_error','r2'], cv=ps_4, refit=False)
    grid_svm_4.fit(x4_scale[i], y_4)

    grid_svm_5 = GridSearchCV(svm_model, param_grid_svm, scoring=['neg_mean_squared_error','r2'], cv=ps_5, refit=False)
    grid_svm_5.fit(x5_scale[i], y_5)
    
    svm_result_list = [grid_svm_1.cv_results_, grid_svm_2.cv_results_, grid_svm_3.cv_results_, grid_svm_4.cv_results_, 
                        grid_svm_5.cv_results_]

    svm_result_dict[feats] = cv_results(svm_result_list)

In [ ]:
with open('svm_results_final.pkl', 'wb') as f:
    pickle.dump(svm_result_dict, f)

$Data Preparation$

In [ ]:
#Reading in test data
test_data = pd.read_csv('final_test_data.csv')
test_data = test_data[['height', 'Sex','LDPRED2_0.841982_0.3267_TRUE', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']]
test_data.columns = ['height', 'Sex', 'full_prs', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
test_data.drop(['height', 'Sex', 'full_prs'], axis=1, inplace=True)
test_gene_sets = pd.read_csv('test_data_set_scores.csv')
full_test_data = pd.concat([test_data, test_gene_sets], axis=1)

In [ ]:
#Reading in training data
train_data = pd.read_csv('full_train_set.csv')
train_data = train_data[['height', 'Sex','LDPRED2_0.841982_0.3267_TRUE', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']]
train_data.columns = ['height', 'Sex', 'full_prs', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
train_data.drop(['height', 'Sex','full_prs'], axis=1, inplace=True)
train_gene_sets = pd.read_csv('set_scores.csv')
full_train_data = pd.concat([train_data, train_gene_sets], axis=1)

In [ ]:
#Defining target and features
X_train, y_train = full_train_data.iloc[:,:-1], full_train_data.iloc[:,-1]
X_test, y_test = full_test_data.iloc[:,:-1], full_test_data.iloc[:,-1]

In [ ]:
#Function to evaluate model based on R2 and MSE
def eval_model(y_preds, y_true):
    r2 = r2_score(y_true, y_preds)
    mse = mean_squared_error(y_true, y_preds)
    
    print(f'Test R2 Score: {r2}')
    print(f'Test MSE: {mse}')

In [ ]:
#Function to scale X data
def final_scale_data(x):
    
    scaler = StandardScaler()
    #Rescaling sex
    x['sex'] = x['sex'].apply(lambda x: -1 if x == 1 else 1)
    
    #Scaling remaining features
    x_new = scaler.fit_transform(x.iloc[:,:70])
    x_new = pd.DataFrame(data=x_new, columns=scaler.get_feature_names_out() )
    
    #Joining datasets
    x_new = pd.concat([x_new, x.iloc[:,-1]], axis=1)
    
    return x_new 

In [ ]:
#Scaled X data
X_train_scaled = final_scale_data(X_train)
X_test_scaled = final_scale_data(X_test)

$RuleFit$

In [ ]:
import rulefit_enet as rf_enet

In [ ]:
#5-fold Cross-Validation of Rule Fit Model
rfit_1 = rf_enet.RuleFit(tree_generator = RandomForestRegressor(n_estimators=500), cv=ps_1)
rfit_1.fit(x_1.values, y_1, feature_names=x_1.columns)

rfit_2 = rf_enet.RuleFit(tree_generator = RandomForestRegressor(n_estimators=500), cv=ps_2)
rfit_2.fit(x_2.values, y_2, feature_names=x_2.columns)

rfit_3 = rf_enet.RuleFit(tree_generator = RandomForestRegressor(n_estimators=500), cv=ps_3)
rfit_3.fit(x_3.values, y_3, feature_names=x_3.columns)

rfit_4 = rf_enet.RuleFit(tree_generator = RandomForestRegressor(n_estimators=500), cv=ps_4)
rfit_4.fit(x_4.values, y_4, feature_names=x_4.columns)

rfit_5 = rf_enet.RuleFit(tree_generator = RandomForestRegressor(n_estimators=500), cv=ps_5)
rfit_5.fit(x_5.values, y_5, feature_names=x_5.columns)

In [ ]:
#Creating dataframe of 5-fold CV results for RuleFit model
num_alphas = rfit_1.enetcv.mse_path_.shape[1]
num_l1_ratio = rfit_1.enetcv.mse_path_.shape[0]
l1_ratios = [.1, .5, .7, .9, .95, .99, 1]
alphas = rfit_1.enetcv.alphas_
rfit_res_dict = dict(keys=['fold_1', 'fold_2', 'fold_3', 'fold_4', 'fold_5', 'alpha', 'l1_ratio'])
f1, f2, f3, f4, f5 = [], [], [], [], []
alpha_vals, l1_ratio_vals = [],[]

for j in range(num_l1_ratio):
    for i in range(num_alphas):
        alpha_vals.append(alphas[j][i])
        l1_ratio_vals.append(l1_ratios[j])
        f1.append(rfit_1.enetcv.mse_path_[j][i])
        f2.append(rfit_2.enetcv.mse_path_[j][i])
        f3.append(rfit_3.enetcv.mse_path_[j][i])
        f4.append(rfit_4.enetcv.mse_path_[j][i])
        f5.append(rfit_5.enetcv.mse_path_[j][i])
    
rfit_res_dict['fold_1'] = f1
rfit_res_dict['fold_2'] = f2
rfit_res_dict['fold_3'] = f3
rfit_res_dict['fold_4'] = f4
rfit_res_dict['fold_5'] = f5
rfit_res_dict['alpha'] = alpha_vals
rfit_res_dict['l1_ratio'] = l1_ratio_vals

rfit_df = pd.DataFrame(columns=['fold_1', 'fold_2', 'fold_3', 'fold_4', 'fold_5', 'alpha', 'l1_ratio'], data=rfit_res_dict)
rfit_df['mean_mse'] = rfit_df.apply(lambda x: np.mean([x.fold_1,x.fold_2,x.fold_3,x.fold_4,x.fold_5]),axis=1)
rfit_df['sd_mse'] = rfit_df.apply(lambda x: np.std([x.fold_1,x.fold_2,x.fold_3,x.fold_4,x.fold_5]),axis=1)

In [ ]:
#Determining optimal value of alpha from 5-fold CV
min_rfit_id = np.argmin(rfit_df['mean_mse'])
alpha_best = rfit_df['alpha'][min_rfit_id]
rfit_df.iloc[min_rfit_id,:]

In [ ]:
#Rule fit model using elastic net regression
rfit_enet = rf_enet.RuleFit(tree_generator = RandomForestRegressor(n_estimators=100, max_features=4, max_depth=10, min_samples_leaf=1, min_samples_split=2), cv=5)
rfit_enet.fit(X_train.values, y_train, feature_names=X_train.columns)

In [ ]:
#Extracting and exporting RuleFit features for plotting in R
rfit_rules = rfit_enet.get_rules().sort_values(by='importance', ascending=False)
rfit_rules.to_csv('rfit_rules.csv')

In [ ]:
#Extracting results for optimal parameter values
opt_alpha = rfit_enet.enetcv.alpha_
opt_lambda = rfit_enet.enetcv.l1_ratio_
l1_ratios = [.1, .5, .7, .9, .95, .99, 1]
lambda_id = np.where(l1_ratios == opt_lambda)
alpha_id = np.where(rfit_enet.enetcv.alphas_[lambda_id] == opt_alpha)

In [ ]:
#MSE values for best parameter set
mse_list = rfit_enet.enetcv.mse_path_[lambda_id][alpha_id]

In [ ]:
#Mean and SD MSE
mean_mse_rfit = np.mean(mse_list)
sd_mse_rfit = np.std(mse_list)

$Cross-Validation Results$

In [ ]:
#Reading in results from models
with open("svm_results_final.pkl", "rb") as f:
    svm_results = pickle.load(f)
    
with open("rf_results_final.pkl", "rb") as f:
    rf_results = pickle.load(f)
    
with open("rf_results_full_set_final.pkl", "rb") as f:
    rf_full_set_results = pickle.load(f)    
    
with open("xgb_results_final.pkl", "rb") as f:
    xgb_results = pickle.load(f)
    
with open('xgb_results_final_full_set.pkl', 'rb') as f:
    xgb_results_reg = pickle.load(f)

with open("enet_full_set_results_final.pkl", "rb") as f:
    enet_results = pickle.load(f)    

In [ ]:
#Function to return top models ordered by R2 for each method
def top_models(result_dict):
    r2_scores = result_dict['mean_r2']
    sd_r2 = result_dict['sd_r2']
    mse_scores = result_dict['mean_neg_mse']
    sd_mse = result_dict['sd_neg_mse']
    params = result_dict['params']
    data_tuple = list(zip(r2_scores, sd_r2, mse_scores, sd_mse, params))
    results_df = pd.DataFrame(data_tuple, columns=['mean_r2', 'sd_r2', 'mean_mse', 'sd_mse', 'params'])
    results_df.sort_values(by='mean_r2', inplace=True, ascending=False)
    
    return results_df

In [ ]:
#Creating dataframes of top 5 models for each method
enet_df = top_models(enet_results)
xgb_reg_df = top_models(xgb_results_reg)
xgb_df = top_models(xgb_results['10'])
rf_full_set_df = top_models(rf_full_set_results)
rf_df = top_models(rf_results['10'])
svm_df = top_models(svm_results['10'])

In [ ]:
#Elastic Net 5-fold CV R2
print(f'Elastic Net \nR2: {max(enet_results["mean_r2"])}\n')

In [ ]:
#Random Forest 5-fold CV R2
for feat in num_feats:
    print(f'Features: {feat}\nR2: {max(rf_results[feat]["mean_r2"])}\n')

In [ ]:
#XGBoost-reg 5-fold CV R2
print(f'R2: {max(xgb_results_reg["mean_r2"])}\n')

In [ ]:
#XGBoost-feat 5-fold CV R2
for feat in num_feats:
    print(f'Features: {feat}\nR2: {max(xgb_results[feat]["mean_r2"])}\n')

In [ ]:
#SVR 5-fold CV R2
for feat in num_feats:
    print(f'Features: {feat}\nR2: {max(svm_results[feat]["mean_r2"])}\n')

In [ ]:
#Creating dataframe of model R2 scores for each number of features
model_list = ['XGB', 'RF', 'SVR']
result_list = [xgb_results, rf_results, svm_results]

models = []
feats = []
r2_vals = []
r2_sd = []

for i, model in enumerate(result_list):
    for j, feat in enumerate(num_feats):
        models.append(model_list[i])
        feats.append(feat)
        r2_vals.append(max(model[feat]['mean_r2']))
        r2_sd.append(model[feat]['sd_r2'][np.argmax(model[feat]['mean_r2'])])
    
feat_res_df = pd.DataFrame(list(zip(models, feats, r2_vals, r2_sd)), columns=['model','num_features','r2', 'sd'])
feat_res_df.to_csv('results_feat_num.csv')

In [ ]:
#Doing feature selection for full training set
top_feats_ordered = mrmr_reg(X_train, y_train, K=30)

$Test Set Evaluation$

In [ ]:
#Random Forest
rf_best = RandomForestRegressor(n_estimators=100, criterion='squared_error', max_depth=50,
                               min_samples_leaf=1, min_samples_split=10, max_features=6,
                               random_state=random_seed)

rf_best.fit(X_train.loc[:,top_feats_ordered[:10]], y_train)

rf_preds = rf_best.predict(X_test.loc[:,top_feats_ordered[:10]])

eval_model(rf_preds, y_test)

In [ ]:
#XGBoost-Reg
xgb_best_reg = XGBRegressor(colsample_bytree=0.8, gamma=0.5, learning_rate=0.01,
                               max_depth=3, min_child_weight=6, n_estimators=1000,
                        n_jobs=-1, objective='reg:squarederror', subsample=0.5,
                        reg_alpha=10, reg_lambda=10,
                        random_state=random_seed)

xgb_best_reg.fit(X_train, y_train)

xgb_preds_reg = xgb_best_reg.predict(X_test)

eval_model(xgb_preds_reg,y_test)

In [ ]:
#XGBoost-Feat
xgb_best= XGBRegressor(colsample_bytree=0.8, gamma=0.5, learning_rate=0.01,
                               max_depth=3, min_child_weight=3, n_estimators=1000,
                        n_jobs=-1, objective='reg:squarederror', subsample=0.8,
                        random_state=random_seed)

xgb_best.fit(X_train.loc[:,top_feats_ordered[:10]], y_train)

xgb_preds = xgb_best.predict(X_test.loc[:,top_feats_ordered[:10]])

eval_model(xgb_preds,y_test)

In [ ]:
#Elastic Net
enet_best = ElasticNet(alpha=0.1, l1_ratio=0.1)

enet_best.fit(X_train, y_train)

enet_preds = enet_best.predict(X_test)

eval_model(enet_preds, y_test)

In [ ]:
#SVM - Linear
svm_best = SVR(C=10, degree=2, gamma='auto', kernel='linear', epsilon=5)

svm_best.fit(X_train_scaled.loc[:,top_feats_ordered[:10]], y_train)

svm_preds = svm_best.predict(X_test_scaled.loc[:,top_feats_ordered[:10]])

eval_model(svm_preds,y_test)

In [ ]:
#SVM - RBF
svm_rbf = SVR(C=10,gamma='auto', kernel='rbf', epsilon=5)

svm_rbf.fit(X_train_scaled.loc[:,top_feats_ordered[:10]], y_train)

svm_rbf = svm_rbf.predict(X_test_scaled.loc[:,top_feats_ordered[:10]])

eval_model(svm_rbf,y_test)

In [ ]:
#SVM - Polynomial
svm_poly = SVR(C=10, degree=3, gamma='auto', kernel='poly', epsilon=5)

svm_poly.fit(X_train_scaled.loc[:,top_feats_ordered[:10]], y_train)

svm_poly = svm_poly.predict(X_test_scaled.loc[:,top_feats_ordered[:10]])

eval_model(svm_poly,y_test)

In [ ]:
#RuleFit
rule_preds = rfit_enet.predict(X_test)

eval_model(rule_preds, y_test)

In [ ]:
#Exporting Model Predictions for Williams Test
pred_df = pd.DataFrame(columns=['pred_enet','pred_xgb','pred_svm','pred_rf','actual'])
pred_df['pred_enet'] = enet_preds
pred_df['pred_xgb'] = xgb_preds_reg
pred_df['pred_svm'] = svm_preds
pred_df['pred_rf'] = rf_preds
pred_df['pred_rfit'] = rfit_preds_enet
pred_df['actual'] = y_test
pred_df.to_csv('all_preds.csv')